In [11]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit import als
from implicit.evaluation import mean_average_precision_at_k
import os


In [12]:
os.environ["MKL_NUM_THREADS"] = "1"

In [13]:

# Membaca dataset
train_df = pd.read_csv('D:/SEM 7 TA/Sistem Rekomendasi/proyek/proyek sisrek/train.csv')


In [14]:

# Menambahkan kolom interaction dengan nilai 1 (karena interaksi hanya 1)
train_df['interaction'] = 1


In [15]:
# Membuat mapping untuk user_id dan item_id
user_mapping = {user: idx for idx, user in enumerate(train_df['user_id'].unique())}
item_mapping = {item: idx for idx, item in enumerate(train_df['item_id'].unique())}


In [16]:
# Mengganti user_id dan item_id dengan indeks
train_df['user_idx'] = train_df['user_id'].map(user_mapping)
train_df['item_idx'] = train_df['item_id'].map(item_mapping)


In [17]:

# Membuat sparse matrix (user-item matrix)
sparse_matrix = csr_matrix((train_df['interaction'], (train_df['user_idx'], train_df['item_idx'])))


In [18]:
# Membangun model ALS
model = als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=30, random_state=42)

In [19]:

# Melatih model ALS
model.fit(sparse_matrix)


  0%|          | 0/30 [00:00<?, ?it/s]

In [20]:
def get_recommendations(user_id, model, sparse_matrix, num_recommendations=10):
    user_idx = user_id
    results = model.recommend(user_idx, sparse_matrix[user_idx], N=num_recommendations)

    if isinstance(results, tuple) and len(results) == 2:
        item_ids, scores = results
        recommended_items = list(item_ids)
    else:
        recommended_items = [item for item, score in results]

    return recommended_items


In [21]:
print(type(get_recommendations))
print(get_recommendations)


<class 'function'>
<function get_recommendations at 0x000002D21227DEE0>


In [22]:
user_id = 1
recommended_items = get_recommendations(user_id, model, sparse_matrix, num_recommendations=10)
print(f"Rekomendasi untuk pengguna {user_id}: {recommended_items}")

Rekomendasi untuk pengguna 1: [366, 351, 18, 154, 538, 1069, 131, 531, 2080, 138]


In [31]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

# Pisahkan sebagian interaksi tiap user untuk test
def train_test_split_per_user(df, test_ratio=0.2):
    test_list = []
    train_list = []
    for user_id, group in df.groupby('user_idx'):
        if len(group) > 1:
            test_size = max(1, int(len(group) * test_ratio))
            test_sample = group.sample(test_size, random_state=42)
            train_sample = group.drop(test_sample.index)
            test_list.append(test_sample)
            train_list.append(train_sample)
        else:
            train_list.append(group)
    return pd.concat(train_list), pd.concat(test_list)

train_df_split, test_df_split = train_test_split_per_user(train_df)

# Buat sparse matrix baru
train_matrix = csr_matrix((train_df_split['interaction'], (train_df_split['user_idx'], train_df_split['item_idx'])))
test_matrix = csr_matrix((test_df_split['interaction'], (test_df_split['user_idx'], test_df_split['item_idx'])))

# Latih ulang model
model.fit(train_matrix)

# Hitung MAP
map_score = mean_average_precision(model, test_df_split, train_matrix, k=10, sample_users=100)
print(f"Mean Average Precision (MAP@10): {map_score:.4f}")


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 187.23it/s]

Mean Average Precision (MAP@10): 0.0111


In [32]:
user = np.random.choice(train_df['user_idx'].unique())
rec_items, scores = model.recommend(user, sparse_matrix[user], N=10)
print("User:", user)
print("Recommended items:", rec_items)
print("Actual items:", train_df[train_df['user_idx']==user]['item_idx'].tolist())


User: 1019
Recommended items: [ 1892   531   669   191  4711 15822 20807  4324   703   845]
Actual items: [8406, 19093, 19094, 19095, 19096, 19097, 7029, 19098, 19099, 19100, 19101, 19102, 19103, 19104, 19105, 19106, 19107, 19108, 9258, 19109, 19110, 19111, 19112, 11934]
